# Assignment 1 — Statistical Learning Foundations
Advanced Machine and Deep Learning (WS 25/26)

This notebook contains complete solutions for Q1–Q4. Each section mirrors the assignment wording and includes the requested plots and metrics. I kept the code explicit and commented so it’s easy to follow or adapt.

## Environment & Imports
We use only standard scientific Python libraries. Plots are made with matplotlib (one chart per figure, default styles).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, Dict

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso
from sklearn.datasets import fetch_california_housing, make_regression

np.random.seed(42)
plt.rcParams['figure.figsize'] = (7, 4)


---
## Q1. Estimators — Sample Mean Convergence
**Task:** Generate samples $x_i \sim \mathcal N(0,1)$ for $n \in \{10, 100, 1000, 10000, 100000\}$ and plot how the sample mean changes as $n$ increases. Include the true mean as a horizontal line.

In [ ]:
ns = [10, 100, 1000, 10000, 100000]
sample_means = []
for n in ns:
    x = np.random.randn(n)
    sample_means.append(np.mean(x))
sample_means = np.array(sample_means)

plt.figure()
plt.plot(ns, sample_means, marker='o')
plt.axhline(0.0)
plt.xscale('log')
plt.xlabel('n (log scale)')
plt.ylabel('sample mean')
plt.title('Sample mean vs. n for N(0,1)')
plt.show()

for n, m in zip(ns, sample_means):
    print(f"n={n:6d} -> sample mean = {m:+.6f}")


---
## Q2. Regression with Gaussian Noise (MLE)
Model: $y_i = x_i^\top \theta + \varepsilon_i$, with $\varepsilon_i \sim \mathcal N(0,\sigma^2)$. For this model,
$$\hat\theta = (X^\top X)^{-1} X^\top y,$$
after standardizing input features (not the target). We:
1. Load **CaliforniaHousing** (with an offline fallback if needed).
2. Standardize features.
3. Fit the closed-form linear model.
4. Report training and testing MSE.
5. Plot a **learning curve** (train/test MSE vs. training fraction).
6. Verify the **MLE–MSE equivalence** by comparing per-sample NLL and MSE using a fixed $\hat\sigma^2$ from the full training residuals.

In [ ]:
def load_california_or_synthetic(random_state: int = 42) -> Tuple[np.ndarray, np.ndarray, Dict]:
    """Try CaliforniaHousing; if unavailable, use a synthetic regression dataset of same shape.
    Returns X, y, meta with feature_names and source tag.
    """
    try:
        data = fetch_california_housing()
        X = data.data
        y = data.target
        meta = {"source": "CaliforniaHousing", "feature_names": list(data.feature_names)}
        return X, y, meta
    except Exception:
        # Synthetic fallback: same n_samples and n_features; scale y to ~[0,5]
        X, y = make_regression(n_samples=20640, n_features=8, noise=12.0, random_state=random_state)
        y = (y - y.min()) / (y.max() - y.min()) * 5.0
        meta = {"source": "SyntheticFallback", "feature_names": [f"f{i}" for i in range(8)]}
        return X, y, meta

X, y, meta = load_california_or_synthetic()
X.shape, y.shape, meta


### Q2(a). Closed-form fit with standardized inputs
We split into train/test (80/20), standardize inputs using `StandardScaler` (fitted on train only), compute the closed-form solution, and report MSE.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler().fit(X_train)
Xs_train = scaler.transform(X_train)
Xs_test = scaler.transform(X_test)

def add_intercept(A: np.ndarray) -> np.ndarray:
    return np.hstack([np.ones((A.shape[0], 1)), A])

def closed_form_theta(Xs: np.ndarray, y: np.ndarray) -> np.ndarray:
    Xb = add_intercept(Xs)
    theta = np.linalg.pinv(Xb.T @ Xb) @ (Xb.T @ y)
    return theta

theta_cf = closed_form_theta(Xs_train, y_train)

def predict_theta(theta: np.ndarray, Xs: np.ndarray) -> np.ndarray:
    return add_intercept(Xs) @ theta

y_pred_train = predict_theta(theta_cf, Xs_train)
y_pred_test = predict_theta(theta_cf, Xs_test)
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Source: {meta['source']}")
print(f"Training MSE: {mse_train:.6f}")
print(f"Testing  MSE: {mse_test:.6f}")


### Q2(b). Learning curve (train/test MSE vs. training fraction)
We randomly shuffle the training data, then train on fractions from 10% to 100% and compute train/test MSE for each fraction.

In [ ]:
fractions = np.linspace(0.1, 1.0, 10)
idx = np.random.permutation(len(Xs_train))

mse_tr_curve, mse_te_curve = [], []
for f in fractions:
    k = max(2, int(len(Xs_train) * f))
    X_sub = Xs_train[idx[:k]]
    y_sub = y_train[idx[:k]]
    th = closed_form_theta(X_sub, y_sub)
    mse_tr_curve.append(mean_squared_error(y_sub, predict_theta(th, X_sub)))
    mse_te_curve.append(mean_squared_error(y_test, predict_theta(th, Xs_test)))

plt.figure()
plt.plot(fractions, mse_tr_curve, marker='o', label='Train MSE')
plt.plot(fractions, mse_te_curve, marker='s', label='Test MSE')
plt.xlabel('Training fraction')
plt.ylabel('MSE')
plt.title('Learning curve: MSE vs. training fraction')
plt.legend()
plt.show()


### Q2(c). Verify MLE–MSE equivalence
We first estimate a fixed variance $\hat\sigma^2$ from the residuals of the **full training set** model (using all training data), then for each training fraction compute the **per-sample** Gaussian NLL and MSE. Since
$$\text{NLL}(\theta) = \frac{1}{2\hat\sigma^2} \sum_i (y_i - x_i^\top\theta)^2 + \text{const},$$
the two curves should have the same shape, differing only by a scale factor.


In [ ]:
# Fixed sigma^2 from full training residuals
res_full = y_train - predict_theta(theta_cf, Xs_train)
sigma2_hat = float(np.var(res_full, ddof=1))  # unbiased estimate
print(f"Estimated sigma^2 from full training residuals: {sigma2_hat:.6f}")

per_sample_mse = []
per_sample_nll = []
for f, mse_tr in zip(fractions, mse_tr_curve):
    # per-sample MSE is already mean of squared residuals
    per_sample_mse.append(mse_tr)
    # per-sample NLL (ignoring constants): (1/(2*sigma^2)) * MSE
    per_sample_nll.append(0.5 * mse_tr / sigma2_hat)

plt.figure()
plt.plot(fractions, per_sample_mse, marker='o', label='Per-sample MSE')
plt.plot(fractions, per_sample_nll, marker='s', label='Per-sample NLL (scaled MSE)')
plt.xlabel('Training fraction')
plt.ylabel('Value (arbitrary scale)')
plt.title('MSE vs. NLL (fixed sigma^2): identical shapes up to scale')
plt.legend()
plt.show()


---
## Q3. MAP Estimation & Regularization — Ridge vs. Lasso
Under a Gaussian prior $\theta \sim \mathcal N(0, \tau^2 I)$, the MAP solution adds an L2 penalty (Ridge) with $\lambda = \sigma^2/\tau^2$. With a Laplace prior, the MAP solution adds an L1 penalty (Lasso).

**Task:** Standardize inputs, keep target unchanged. For $\lambda \in \{10^{-6},10^{-5},\dots,10^{3}\}$, fit Ridge and Lasso and compute **training** and **testing** **RMSE** for each $\lambda$. We report results and plot RMSE vs. $\log_{10} \lambda$. 

In [ ]:
lambdas = np.array([10.0**k for k in range(-6, 4)])  # 1e-6 ... 1e3

ridge_train_rmse, ridge_test_rmse = [], []
lasso_train_rmse, lasso_test_rmse = [], []

for lam in lambdas:
    # Ridge
    ridge = Ridge(alpha=lam)
    ridge.fit(Xs_train, y_train)
    r_tr = np.sqrt(mean_squared_error(y_train, ridge.predict(Xs_train)))
    r_te = np.sqrt(mean_squared_error(y_test, ridge.predict(Xs_test)))
    ridge_train_rmse.append(r_tr)
    ridge_test_rmse.append(r_te)

    # Lasso (increase max_iter for stability)
    lasso = Lasso(alpha=lam, max_iter=20000)
    lasso.fit(Xs_train, y_train)
    l_tr = np.sqrt(mean_squared_error(y_train, lasso.predict(Xs_train)))
    l_te = np.sqrt(mean_squared_error(y_test, lasso.predict(Xs_test)))
    lasso_train_rmse.append(l_tr)
    lasso_test_rmse.append(l_te)

ridge_train_rmse = np.array(ridge_train_rmse)
ridge_test_rmse = np.array(ridge_test_rmse)
lasso_train_rmse = np.array(lasso_train_rmse)
lasso_test_rmse = np.array(lasso_test_rmse)

print("Ridge best test RMSE:", float(ridge_test_rmse.min()), "at lambda=", float(lambdas[ridge_test_rmse.argmin()]))
print("Lasso best test RMSE:", float(lasso_test_rmse.min()), "at lambda=", float(lambdas[lasso_test_rmse.argmin()]))


### Plots: RMSE vs $\log_{10}(\lambda)$
One figure for Ridge, one for Lasso, training and testing curves together.

In [ ]:
logl = np.log10(lambdas)

plt.figure()
plt.plot(logl, ridge_train_rmse, marker='o', label='Train RMSE')
plt.plot(logl, ridge_test_rmse, marker='s', label='Test RMSE')
plt.xlabel('log10(lambda)')
plt.ylabel('RMSE')
plt.title('Ridge: RMSE vs log10(lambda)')
plt.legend()
plt.show()

plt.figure()
plt.plot(logl, lasso_train_rmse, marker='o', label='Train RMSE')
plt.plot(logl, lasso_test_rmse, marker='s', label='Test RMSE')
plt.xlabel('log10(lambda)')
plt.ylabel('RMSE')
plt.title('Lasso: RMSE vs log10(lambda)')
plt.legend()
plt.show()


---
## Q4. Information & Cross-Entropy — KL Divergence under Drift
We construct a discrete base distribution $p$ over indices $\{0,1,\dots,20\}$ with a Gaussian shape centered at 10, then create shifted distributions $q_\Delta$ with center moved by $\Delta \in \{-8,-6,\dots,8\}$. We plot:
1. The base $p$.
2. A representative comparison of $p$ vs. $q_\Delta$ (e.g., $\Delta=5$).
3. The KL divergence $D_{\mathrm{KL}}(p\Vert q_\Delta)$ vs. $\Delta$.

All distributions are normalized; we add a small $\varepsilon$ to avoid division by zero in the KL computation.

In [ ]:
idx = np.arange(21)
center = 10
sigma = 3.0

p = np.exp(-0.5 * ((idx - center)/sigma)**2)
p = p / p.sum()

plt.figure()
plt.stem(idx, p, use_line_collection=True)
plt.xlabel('index')
plt.ylabel('p(i)')
plt.title('Base distribution p over {0..20}')
plt.show()

def shifted_q(delta: int) -> np.ndarray:
    q_center = center + delta
    q = np.exp(-0.5 * ((idx - q_center)/sigma)**2)
    q = q / q.sum()
    return q

delta_example = 5
q_ex = shifted_q(delta_example)

plt.figure()
plt.plot(idx, p, marker='o', label='p (center=10)')
plt.plot(idx, q_ex, marker='s', label=f'q_Δ (center=10+{delta_example})')
plt.xlabel('index')
plt.ylabel('probability')
plt.title('p vs q_Δ (representative drift)')
plt.legend()
plt.show()

def kl_divergence(p: np.ndarray, q: np.ndarray, eps: float = 1e-12) -> float:
    p_safe = np.clip(p, eps, 1.0)
    q_safe = np.clip(q, eps, 1.0)
    return float(np.sum(p_safe * np.log(p_safe / q_safe)))

deltas = np.arange(-8, 9, 2)
kls = []
for d in deltas:
    kls.append(kl_divergence(p, shifted_q(d)))
kls = np.array(kls)

plt.figure()
plt.plot(deltas, kls, marker='o')
plt.xlabel('Δ (drift)')
plt.ylabel('DKL(p || q_Δ)')
plt.title('KL divergence vs drift Δ')
plt.show()

for d, val in zip(deltas, kls):
    print(f"Δ={d:+d} -> D_KL={val:.6f}")


---
### Notes
- Q2 uses standardized inputs with an explicit intercept term for the closed-form solution. Targets remain unscaled.
- For Q2(c), the per-sample NLL is computed with a **fixed** $\hat\sigma^2$ estimated from the full training residuals, as requested.
- Q3 scans $\lambda$ on a wide log scale for both Ridge and Lasso. Lasso uses a higher `max_iter` for reliable convergence.
- Q4 ensures distributions are normalized and guards KL against zero probabilities with a small $\varepsilon$.
